# Compound model
We want to concatenate the different models.
For example:

Client---[CG]--(RTT:1ms, p = 0.0000001)----Gateway-----------[SAT]---(RTT: 500ms, p=0.0005)----------Terminal----[TS]---(RTT: 25ms, p = 0.0000001)-------Server

vs 

Client---[CS]--(RTT: 526ms,p=0.0005)-------Server

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging, sys
from ccaModels import CCA_MarkovChain_CUBIC_OG, CCA_MarkovChain_Hybla_discrete, CCA_MarkovChain_Hybla_OG, CCA_MarkovChain_CUBIC_discrete
from ccaModels import ISP_Compound

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
C = 1000
# goodput ratio on the qpep link
goodputRatio = 0.95
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

# CG = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.01,packet_err = 1e-6)
# SAT = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.5,packet_err = 1e-4)
# TS = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.025, packet_err = 1e-6)

CG = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.01,packet_err = 1e-6)
SAT_Hyb = CCA_MarkovChain_Hybla_discrete(RTT_real = 0.5,packet_err = 1e-4, RTT0= 0.025)
SAT_Cub = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.5, packet_err = 1.e-4)
TS = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.025, packet_err = 1e-6)

segmented = ISP_Compound(CG=CG,SAT = SAT_Hyb,TQ = TS, QS = TS, goodputRatio = goodputRatio)
print(segmented)
segmented.compute_throughput()
print("Bottleneck link and throughput",segmented.bottleNeck, segmented.min_throughput)
print("Goodput", segmented.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
segmented = ISP_Compound(CG=CG,SAT = SAT_Hyb,TS = TS, goodputRatio = goodputRatio)
print(segmented)
segmented.compute_throughput()
print("Bottleneck link and throughput",segmented.bottleNeck, segmented.min_throughput)
print("Goodput", segmented.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
segmented_cub = ISP_Compound(CG=CG,SAT = SAT_Cub,TS = TS, goodputRatio = goodputRatio)
print(segmented_cub)
segmented_cub.compute_throughput()
print("Bottleneck link and throughput",segmented_cub.bottleNeck, segmented_cub.min_throughput)
print("Goodput", segmented_cub.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented_cub.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
#Full connection
CS = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.535, packet_err = 1.02e-4)
direct = ISP_Compound(CS = CS)
print(direct)
direct.compute_throughput()
print("Bottleneck link and throughput",direct.bottleNeck,direct.min_throughput)
print("Goodput",direct.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", direct.time_to_transfer(filesize=1000))

Client X-----CG: CUBIC-----X-----SAT: HYBLA-----X-----TQ: CUBIC-----X-----QS: CUBIC-----X Server
Bottleneck link and throughput SAT 844.7437203046401
Goodput 802.506534289408
Time to transfer a 1000MSS file (1Mbyte) 1.2460957727720756
_______________________________________________________________
Client X-----CG: CUBIC-----X-----SAT: HYBLA-----X-----TS: CUBIC-----X Server
Bottleneck link and throughput SAT 844.74372030464
Goodput 802.5065342894079
Time to transfer a 1000MSS file (1Mbyte) 1.2460957727720756
_______________________________________________________________
Client X-----CG: CUBIC-----X-----SAT: CUBIC-----X-----TS: CUBIC-----X Server
Bottleneck link and throughput SAT 732.0206597774842
Goodput 695.4196267886099
Time to transfer a 1000MSS file (1Mbyte) 1.437980697522037
_______________________________________________________________
Client X-----CS: CUBIC-----X Server
Bottleneck link and throughput CS 726.0215934302411
Goodput 726.0215934302411
Time to transfer a 1000MSS fil

In [3]:
# LEO comparison
C = 1000
# goodput ratio on the qpep link
goodputRatio = 0.95
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

# CG = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.01,packet_err = 1e-6)
# SAT = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.5,packet_err = 1e-4)
# TS = CCA_MarkovChain_CUBIC_OG(RTT_real = 0.025, packet_err = 1e-6)

CG = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.01,packet_err = 1e-6)
SAT_Hyb = CCA_MarkovChain_Hybla_discrete(RTT_real = 0.1,packet_err = 1e-4, RTT0= 0.025)
SAT_Cub = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.1, packet_err = 1.e-4)
TS = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.025, packet_err = 1e-6)

segmented = ISP_Compound(CG=CG,SAT = SAT_Hyb,TQ = TS, QS = TS, goodputRatio = goodputRatio)
print(segmented)
segmented.compute_throughput()
print("Bottleneck link and throughput",segmented.bottleNeck, segmented.min_throughput)
print("Goodput", segmented.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
segmented = ISP_Compound(CG=CG,SAT = SAT_Hyb,TS = TS, goodputRatio = goodputRatio)
print(segmented)
segmented.compute_throughput()
print("Bottleneck link and throughput",segmented.bottleNeck, segmented.min_throughput)
print("Goodput", segmented.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
segmented_cub = ISP_Compound(CG=CG,SAT = SAT_Cub,TS = TS, goodputRatio = goodputRatio)
print(segmented_cub)
segmented_cub.compute_throughput()
print("Bottleneck link and throughput",segmented_cub.bottleNeck, segmented_cub.min_throughput)
print("Goodput", segmented_cub.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", segmented_cub.time_to_transfer(filesize=1000))

print("_______________________________________________________________")
#Full connection
CS = CCA_MarkovChain_CUBIC_discrete(RTT_real = 0.135, packet_err = 1.02e-4)
direct = ISP_Compound(CS = CS)
print(direct)
direct.compute_throughput()
print("Bottleneck link and throughput",direct.bottleNeck,direct.min_throughput)
print("Goodput",direct.goodput())
print("Time to transfer a 1000MSS file (1Mbyte)", direct.time_to_transfer(filesize=1000))

Client X-----CG: CUBIC-----X-----SAT: HYBLA-----X-----TQ: CUBIC-----X-----QS: CUBIC-----X Server
Bottleneck link and throughput SAT 844.667911794796
Goodput 802.4345162050562
Time to transfer a 1000MSS file (1Mbyte) 1.246207609225595
_______________________________________________________________
Client X-----CG: CUBIC-----X-----SAT: HYBLA-----X-----TS: CUBIC-----X Server
Bottleneck link and throughput SAT 844.667911794796
Goodput 802.4345162050562
Time to transfer a 1000MSS file (1Mbyte) 1.246207609225595
_______________________________________________________________
Client X-----CG: CUBIC-----X-----SAT: CUBIC-----X-----TS: CUBIC-----X Server
Bottleneck link and throughput SAT 796.1667895032477
Goodput 756.3584500280854
Time to transfer a 1000MSS file (1Mbyte) 1.3221244503355092
_______________________________________________________________
Client X-----CS: CUBIC-----X Server
Bottleneck link and throughput CS 783.4946736012189
Goodput 783.4946736012189
Time to transfer a 1000MSS fil